In [3]:
from bs4 import BeautifulSoup 
import pandas as pd

def extract_matches_betfirst(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    match_data = []
    
    # Debug 1: Vérifier la structure générale
    matches = soup.find_all('div', class_='rj-ev-list__ev-card')
    print(f"\n1. Nombre de cartes de match trouvées: {len(matches)}")
    
    # Debug 2: Examiner une carte de match
    if matches:
        print("\n2. Structure d'une carte de match:")
        print(matches[0].prettify())
    
    for i, match in enumerate(matches):
        print(f"\n3. Analyse du match #{i+1}")
        try:
            # Debug 3: Vérifier les équipes
            teams = match.find_all('span', class_='rj-ev-list__name-text')
            print(f"  - Nombre d'équipes trouvées: {len(teams)}")
            if teams:
                print(f"  - Équipes: {[team.text.strip() for team in teams]}")
            
            # Debug 4: Vérifier la section money line
            ml_section = match.find('div', {'data-market-type': '1_0'})
            print(f"  - Money line section trouvée: {ml_section is not None}")
            
            if ml_section:
                print("\n4. Structure de la section money line:")
                print(ml_section.prettify())
            
            # Debug 5: Vérifier les cotes
            if ml_section:
                odds = ml_section.find_all('span', class_='rj-ev-list__bet-btn__odd')
                print(f"  - Nombre de cotes trouvées: {len(odds)}")
                if odds:
                    print(f"  - Cotes: {[odd.text.strip() for odd in odds]}")
            
            # Tenter d'extraire les données
            if len(teams) >= 2 and ml_section and len(odds) >= 2:
                match_info = {
                    'Home_Team': teams[0].text.strip(),
                    'Away_Team': teams[1].text.strip(),
                    'Home_Odds': float(odds[0].text.strip()),
                    'Away_Odds': float(odds[1].text.strip())
                }
                match_data.append(match_info)
                print("  ✓ Match extrait avec succès")
            else:
                print("  ✗ Données manquantes pour ce match")
                
        except Exception as e:
            print(f"  ✗ Erreur lors de l'extraction: {str(e)}")
            continue

    df = pd.DataFrame(match_data)
    
    print("\n5. Résultat final:")
    print(f"  - Matches extraits: {len(df)}")
    if not df.empty:
        print("\nDonnées extraites:")
        print(df)
    
    return df